## Objetive

Automate the daily process of updating the 'E-commerce Orders' excel database. We need to download the latest order data from the ERP website and merge it with our current database. 

In [1]:
import pandas as pd
import os
import shutil
import time
from datetime import date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
# capture current date

today = date.today()
month = today.strftime("%m")
year = today.year

# create variables of month and year

month_year = str(month) + "/" + str(year) # date format dd/yyyy
year_month = str(year) + "-" + str(month) # date format yyyy-mm

In [3]:
# change the chrome webdriver download folder

options = webdriver.ChromeOptions()
prefs = {'download.default_directory':r'C:\Users\Guilherme\Downloads\Bases_Selenium'}
options.add_experimental_option('prefs', prefs)

### Download the files

In [4]:
# initiate webdriver

nav = webdriver.Chrome(options=options)
nav.maximize_window()

In [5]:
# access ERP website

nav.get("https://tiny.com.br/login")

In [6]:
# insert username

nav.find_element_by_xpath('/html/body/section/div/form/div[1]/div[2]/div[2]/input').send_keys("****@****")

In [7]:
# insert password

nav.find_element_by_xpath('/html/body/section/div/form/div[1]/div[2]/div[3]/input').send_keys("*******")

In [8]:
# log in

nav.find_element_by_xpath('/html/body/section/div/form/div[1]/div[2]/div[3]/input').send_keys(Keys.ENTER)

In [9]:
time.sleep(5)

In [10]:
# personalize the report link with current month

link =  "https://erp.tiny.com.br/exportacao_planilha_nf?tipo=S&m=" + month_year + "&modelo=55"

Now that we logged in, we need to acess the report page and download report for current month


In [11]:
nav.get(link)

In [12]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[3]/div/div/button').click()

In [13]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[3]/div/div/ul/li[1]/a/div/label').click()

In [14]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[5]/div/div/label').click()

In [15]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[6]/div/div[2]/label').click()

In [16]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[8]/button').click()

In [17]:
time.sleep(3)

In [18]:
# identify file name and rename it

filename = max([r'C:\Users\Guilherme\Downloads\Bases_Selenium' + "\\" + f for f in os.listdir(r'C:\Users\Guilherme\Downloads\Bases_Selenium')],key=os.path.getctime)
shutil.move(filename,os.path.join(r'C:\Users\Guilherme\Downloads\Bases_Selenium',r"base_sp.csv"))

'C:\\Users\\Guilherme\\Downloads\\Bases_Selenium\\base_sp.csv'

Logout from the São Paulo branch.

In [19]:
nav.find_element_by_xpath('/html/body/div[3]/div[2]/div[1]/div[1]/div[2]/ul/li[4]/a/div/div[1]/div').click()

In [20]:
nav.find_element_by_xpath('/html/body/div[3]/div[2]/div[2]/nav[6]/div[3]/a').click()

In [21]:
time.sleep(5)

Repeat the process for the Porto Alegre branch report.

In [22]:
nav.get("https://tiny.com.br/login")

In [23]:
time.sleep(5)

In [24]:
nav.find_element_by_xpath('/html/body/section/div/form/div[1]/div[2]/div[2]/input').send_keys("****@*****")

In [25]:
nav.find_element_by_xpath('/html/body/section/div/form/div[1]/div[2]/div[3]/input').send_keys("*******")

In [26]:
nav.find_element_by_xpath('/html/body/section/div/form/div[1]/div[2]/div[3]/input').send_keys(Keys.ENTER)

In [27]:
time.sleep(10)

In [28]:
nav.get(link)

In [29]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[3]/div/div/button').click()

In [30]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[3]/div/div/ul/li[1]/a/div/label').click()

In [31]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[5]/div/div/label').click()

In [32]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[6]/div/div[2]/label').click()

In [33]:
nav.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div[8]/button').click()

In [34]:
time.sleep(3)

In [35]:
nav.quit()

In [36]:
filename = max([r'C:\Users\Guilherme\Downloads\Bases_Selenium' + "\\" + f for f in os.listdir(r'C:\Users\Guilherme\Downloads\Bases_Selenium')],key=os.path.getctime)
shutil.move(filename,os.path.join(r'C:\Users\Guilherme\Downloads\Bases_Selenium',r"base_poa.csv"))

'C:\\Users\\Guilherme\\Downloads\\Bases_Selenium\\base_poa.csv'

### Database update

Using the two donwloaded files, we will update our orders database.

In [37]:
# read downloaded file with orders from current month - SP branch

df_sp = pd.read_csv(r'C:\Users\Guilherme\Downloads\Bases_Selenium\base_sp.csv')

In [38]:
# read downloaded file with orders from current month - POA branch

df_poa = pd.read_csv(r'C:\Users\Guilherme\Downloads\Bases_Selenium\base_poa.csv')

In [39]:
df_sp['ID Nota'].count()

465

In [40]:
df_poa['ID Nota'].count()

336

In [41]:
# read the Ecommerce Orders database

df_tiny = pd.read_excel(r'C:\Users\Guilherme\Downloads\Ecommerce Orders\fNotasTiny.xlsx')

In [42]:
df_tiny['ID Nota'].count()

3256

In [43]:
# drop the rows of the current month from current database

index = df_tiny[df_tiny['Data emissao'].str.contains(year_month)].index
df_tiny.drop(index, inplace=True)

In [44]:
df_tiny['ID Nota'].count()

2656

In [45]:
# merge current report with downloaded files

concatenated = pd.concat([df_tiny, df_sp, df_poa])

In [46]:
concatenated['ID Nota'].count()

3457

In [47]:
# export updated report so we can use it on our Power BI dashboards

concatenated.to_excel(r'C:\Users\Guilherme\Downloads\Ecommerce Orders\fNotasTiny.xlsx', index=False, sheet_name='Notas fiscais')